# Insert Python dataframe into Azure SQL table

## Import the libraries

In [ ]:
import pyodbc
import pandas

## Open the connection

In [ ]:
server = '<server>.database.windows.net'
database = '<database>'
username = '<username>'
password = '<password>'
driver= '{ODBC Driver 17 for SQL Server}'
conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' +
    server + ';PORT=1433;DATABASE=' + database +
    ';UID=' + username + ';PWD=' + password)

## Create a cursor object

In [ ]:
cursor = conn.cursor()

## Create a table

In [ ]:
# Drop previous table of same name if one exists
cursor.execute("DROP TABLE IF EXISTS movies")

# Create a table
cursor.execute("CREATE TABLE movies "
                "(movie_id INT IDENTITY(1,1) PRIMARY KEY, "
                "movie_title VARCHAR(255) NOT NULL, "
                "released_year INT, "
                "runtime INT NOT NULL, "
                "genre VARCHAR(255), "
                "rating INT NOT NULL, "
                "director VARCHAR(255), "
                "star1 VARCHAR(255), "
                "star2 VARCHAR(255), "
                "number_of_votes INT, "
                "gross INT)")

## Open the file

In [ ]:
fpath = "movies.CSV"
df = pandas.read_csv(fpath, delimiter=";", encoding="utf8")

In [ ]:
df

## Transform columns and change data types

### Runtime

In [ ]:
def convert_runtime(x):
    """
    Convert the string value to int.
    - Remove "min".
    - Convert to int.
    """
    return int(x.split()[0])

In [ ]:
df['Runtime'] = df['Runtime'].apply(convert_runtime)

### Genre

In [ ]:
df['Genre'] = df['Genre'].apply(lambda x: x.split(', ')[0])

### Gross

In [ ]:
def convert_gross(x):
    """
    Convert the string value to int.
    - Remove commas.
    - Convert to int.
    """
    return int(x.replace(',', ''))

In [ ]:
df['Gross'] = df['Gross'].apply(convert_gross)

In [ ]:
df

In [ ]:
df.dtypes

## Insert data in the *movies* table

In [ ]:
for index, row in df.iterrows():
    cursor.execute("INSERT INTO movies "
                   "(movie_title, released_year, runtime, "
                   "genre, rating, director, "
                   "star1, star2, number_of_votes, gross) "
                   "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                   (row['Title'], row['Released_Year'], row['Runtime'], row['Genre'], row['Rating'], 
                   row['Director'], row['Star1'], row['Star2'], row['Number_of_Votes'], row['Gross']))

## Commit the changes

In [ ]:
conn.commit()

## Read data

In [ ]:
# Read data
cursor.execute("SELECT "
    "movie_id, movie_title "
    "FROM movies")
rows = cursor.fetchall()

# Print data
for row in rows:
  print(row)

## Close the connection

In [ ]:
cursor.close()
conn.close()